In [ ]:
%pip install fastf1 --quiet
import requests
import codecs
import json
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual, GridspecLayout, AppLayout
import ipywidgets as widgets

# FastF1 API and plotting tools
import seaborn as sns
from matplotlib import pyplot as plt
import fastf1
import fastf1.plotting
import pandas as pd
import base64
import zlib

fastf1.plotting.setup_mpl(mpl_timedelta_support=False, misc_mpl_mods=False, color_scheme='fastf1')

# Raw Data from livetiming.formula1.com


In [ ]:
static_api="https://livetiming.formula1.com/static/"
season_dropdown = widgets.Dropdown(
    options=[i for i in range(2025, 2018, -1)],
    value=2025,
    description='Season:',
    disabled=False,
)

meeting_dropdown = widgets.Dropdown(
    options=[],
    value=None,
    description='Meeting:',
    disabled=False,
)

session_dropdown = widgets.Dropdown(
    options=[],
    value=None,
    description='Session:',
    disabled=False,
)

channel_dropdown = widgets.Dropdown(
    options=[],
    value=None,
    description='Channel:',
    disabled=False,
)

stream_selector = widgets.SelectionSlider(
    options=[(0, {})],
    value={},
    description='Stream:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout={'width': '1000px'}
)

def update_widgets(season_index=None, meeting_index=None, session_index=None, channel=None):
    # print(season_index['Year'], meeting_index)
    if season_index:
        season_dropdown.value=season_index['Year']
        meeting_dropdown.options=[ (meeting['Name'], meeting) for meeting in season_index['Meetings'][::-1]]
    else:
        season_dropdown.value=None
        meeting_dropdown.options=[]

    # Update meeting_dropdown value and session dropdown options
    if meeting_index and meeting_index['Name'] in [ name for name, _ in meeting_dropdown.options]:
        meeting_dropdown.value=meeting_index
        session_dropdown.options=[ (session['Name'], session) for session in meeting_index['Sessions'][::-1]]
    else:
        meeting_dropdown.value=None
        session_dropdown.options=[]

    # update session_dropdown value and channel dropdown options
    try:
        if session_index and session_index['Name'] in [ name for name, _ in session_dropdown.options]:
            session_dropdown.value=session_index
            response=requests.get(static_api + session_index['Path'] + "Index.json")
            channel_index=json.loads(codecs.decode(response.text.encode(), 'utf-8-sig'))
            channel_dropdown.options=[ (key, value) for key, value in channel_index['Feeds'].items()]
        else:
            session_dropdown.value=None
            channel_dropdown.options=[]
    except Exception as e:
        session_dropdown.value=None

    # check and display channel keyframe
    if channel and channel in [value for _, value in channel_dropdown.options]:
        channel_dropdown.value=channel
        response=requests.get(static_api + session_index['Path'] + channel['StreamPath'])
        stream_frames=response.text.split("\r\n")
        stream_frames=[ (frame[:12], json.loads(frame[12:])) for frame in stream_frames if frame[12:]!="" ]
        # stream_selector.options=[ (str(idx), frame) for idx, frame in enumerate(stream_frames) ]
        stream_selector.options=stream_frames
    else:
        channel_dropdown.value=None
        stream_selector.options=None


def browse_keyframe(season, meeting, session, channel):
    try:
        response=requests.get(static_api + str(season) + "/Index.json")
        season_index=json.loads(codecs.decode(response.text.encode(), 'utf-8-sig'))

        if season and meeting and session and channel:
            response=requests.get(static_api + session['Path'] + channel['KeyFramePath'])
            channel_dict=json.loads(codecs.decode(response.text.encode(), 'utf-8-sig'))
            if type(channel_dict) == str :
                channel_dict = json.loads(zlib.decompress(base64.b64decode(channel_dict), -zlib.MAX_WBITS))
            print(json.dumps(channel_dict, indent=2))

    except Exception as e:
        print(e)
        season_index, meeting_index, session_index, channel_index=None, None, None, None
    finally:
        update_widgets(season_index, meeting_index=meeting, session_index=session, channel=channel)
    return

keyframe_interactive_output = widgets.interactive_output(
    browse_keyframe,
    {
        'season': season_dropdown,
        'meeting': meeting_dropdown,
        'session': session_dropdown,
        'channel': channel_dropdown
    }
)

def browse_stream(stream=None):
    if stream:
        if type(stream) == str :
                stream = json.loads(zlib.decompress(base64.b64decode(stream), -zlib.MAX_WBITS))
        print(json.dumps(stream, indent=2))

stream_interactive_output = widgets.interactive_output(
    browse_stream,
    {
        'stream': stream_selector
    }
)

header=widgets.HBox([season_dropdown, meeting_dropdown, session_dropdown, channel_dropdown, stream_selector])
left_siderbar=widgets.HBox([keyframe_interactive_output], layout=widgets.Layout(height='900px', overflow_y='auto'))
right_siderbar=widgets.VBox([stream_interactive_output], layout=widgets.Layout(height='900px', overflow_y='auto'))

app_layout = AppLayout(
    header=header,
    left_sidebar=left_siderbar,
    right_sidebar=right_siderbar,
    center=None,
    pane_widths=[1, 0, 1],
    pane_heights=[1, '900px', 1]
)

display(
    app_layout
)

# FastF1 API

In [ ]:
from re import S
@interact(year=[i for i in range(2024, 2017, -1)])
def get_event_schedule(year):
    global season
    global event_schedule
    season = year
    event_schedule = fastf1.get_event_schedule(year)
    return event_schedule

In [ ]:
@interact(event=event_schedule['EventName'].to_list())
def get_practice_laps(event):
    session_fp1 = fastf1.get_event(season, event).get_practice(1)
    session_fp1.load()
    laps=session_fp1.laps.pick_quicklaps()
    transformed_laps = laps.copy()
    transformed_laps.loc[:, "LapTime (s)"] = laps["LapTime"].dt.total_seconds()
    transformed_laps[["Team", "Stint", "LapTime (s)"]].groupby(["Team", "Stint"])
    # team_order = (
    #     transformed_laps[["Team", "Stint", "LapTime (s)"]].groupby(["Team", "Stint"])
    #     .median()["LapTime (s)"]
    #     .sort_values()
    #     .index
    # )
    # team_palette = {team: fastf1.plotting.get_team_color(team, session=session_fp1)
    #             for team in team_order}

    # fig, ax = plt.subplots(figsize=(15, 10))
    # sns.boxplot(
    #     data=transformed_laps,
    #     x="Team",
    #     y="LapTime (s)",
    #     hue="Team",
    #     order=team_order,
    #     palette=team_palette,
    #     whiskerprops=dict(color="white"),
    #     boxprops=dict(edgecolor="white"),
    #     medianprops=dict(color="grey"),
    #     capprops=dict(color="white"),
    # )

    # plt.title(session_fp1.name)
    # plt.grid(visible=False)

    # # x-label is redundant
    # ax.set(xlabel=None)
    # plt.tight_layout()
    # plt.show()
    return transformed_laps[["Team", "Stint", "Compound", "LapTime (s)"]].groupby(["Team", "Stint", "Compound"]).median().sort_values(by="LapTime (s)")

Get Next event

In [ ]:
remaining_events=fastf1.get_events_remaining()
remaining_events
# remaining_events[['Country', 'EventName', ]].iloc[0]

In [ ]:
event_schedule = fastf1.get_event_schedule(2024)
# event_schedule[["EventName", "Location"]]
list(zip(event_schedule["EventName"], event_schedule["Location"]))